# GPU Audio Transcription - FAST MODE ⚡
### Optimized for speed with tiny model and smart keyword search

**Instructions:**
1. Upload audio_task_43.mp3 to your Google Drive
2. Enable GPU: Runtime → Change runtime type → T4 GPU
3. Run all cells

**Speed optimizations:**
- Uses 'tiny' Whisper model (5-10x faster, slightly less accurate)
- 10-minute chunks (fewer chunks = less overhead)
- Skips already-transcribed chunks (safe to restart)
- Smart keyword search (looks for 'keyword' word first)
- Forces English transcription for multilingual audio

**Estimated time:** ~15-25 minutes (vs 60-90 with base model)

In [ ]:
# Install Whisper
!pip install -q openai-whisper

In [ ]:
# Check GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠ WARNING: Enable GPU in Runtime → Change runtime type")

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Update this path if needed
audio_file = "/content/drive/MyDrive/audio_task_43.mp3"

if os.path.exists(audio_file):
    print(f"✓ Found: {audio_file}")
    print(f"  Size: {os.path.getsize(audio_file) / (1024**2):.1f} MB")
else:
    print(f"⚠ Searching for audio_task_43.mp3...")
    import subprocess
    result = subprocess.run(['find', '/content/drive/MyDrive', '-name', 'audio_task_43.mp3'],
                          capture_output=True, text=True, timeout=60)
    files = [f for f in result.stdout.strip().split('\n') if f]
    if files:
        audio_file = files[0]
        print(f"✓ Found at: {audio_file}")

In [ ]:
# Configuration - OPTIMIZED FOR SPEED
CHUNK_DURATION_MIN = 10  # 10-minute chunks (larger = faster)
TOTAL_DURATION_MIN = 470  # 7h50min

import whisper
import subprocess
import re
import time

# Create output directories
os.makedirs('audio_chunks', exist_ok=True)
os.makedirs('transcriptions', exist_ok=True)
print("✓ Created output directories: audio_chunks/ and transcriptions/")

# Load TINY model for speed
print("Loading Whisper 'tiny' model (FAST MODE)...")
model = whisper.load_model("tiny")
print(f"✓ Model on {'GPU' if next(model.parameters()).is_cuda else 'CPU'}")

num_chunks = (TOTAL_DURATION_MIN + CHUNK_DURATION_MIN - 1) // CHUNK_DURATION_MIN
print(f"\nWill process {num_chunks} chunks of {CHUNK_DURATION_MIN} minutes each")
print(f"⚡ FAST MODE: Using tiny model + large chunks for 5-10x speedup")
print(f"🔑 Forcing English transcription for multilingual audio")

In [ ]:
# SMART keyword search - looks for "keyword" word first, then validates pattern
ordinal_map = {
    'first': 1, '1st': 1, 'second': 2, '2nd': 2, 'third': 3, '3rd': 3,
    'fourth': 4, '4th': 4, 'fifth': 5, '5th': 5, 'sixth': 6, '6th': 6,
    'seventh': 7, '7th': 7, 'eighth': 8, '8th': 8, 'ninth': 9, '9th': 9,
    'tenth': 10, '10th': 10, 'eleventh': 11, '11th': 11, 'twelfth': 12, '12th': 12,
    'thirteenth': 13, '13th': 13, 'fourteenth': 14, '14th': 14,
    'fifteenth': 15, '15th': 15, 'sixteenth': 16, '16th': 16,
    'seventeenth': 17, '17th': 17, 'eighteenth': 18, '18th': 18,
    'nineteenth': 19, '19th': 19, 'twentieth': 20, '20th': 20
}

def parse_ordinal(ordinal_str):
    """Parse ordinal to number"""
    ordinal_lower = ordinal_str.lower()
    if ordinal_lower in ordinal_map:
        return ordinal_map[ordinal_lower]
    
    # Try to extract pure number
    num_match = re.match(r'(\d+)', ordinal_str)
    if num_match:
        return int(num_match.group(1))
    
    return None

def find_keywords(text):
    """
    SMART SEARCH: Find 'keyword' word first, then validate pattern around it.
    More reliable than searching for full pattern.
    """
    results = []
    
    # Find all occurrences of "keyword" (allowing minor misspellings)
    keyword_pattern = r'\bkey\s*wo?r?d\b'
    
    for match in re.finditer(keyword_pattern, text, re.IGNORECASE):
        # Get context around "keyword" (±150 chars)
        start = max(0, match.start() - 150)
        end = min(len(text), match.end() + 150)
        context = text[start:end]
        
        # Look for pattern: [ordinal] letter [in/of] keyword is [letter], [phonetic]
        pattern = r'(\w+)\s+letter\s+(?:in|of)\s+(?:the\s+)?key\s*wo?r?d\s+is\s+([A-Z])[,.\s]+([A-Za-z]+)'
        
        matches = re.findall(pattern, context, re.IGNORECASE)
        
        for ordinal, letter, phonetic in matches:
            pos_num = parse_ordinal(ordinal)
            if pos_num:
                results.append((pos_num, letter.upper(), phonetic))
        
        # If no match with pattern, show context for manual inspection
        if not matches:
            print(f"    ⚠ Found 'keyword' but pattern unclear:")
            print(f"      ...{context[max(0, match.start()-start-50):match.end()-start+50]}...")
    
    return results

print("✓ Smart keyword search ready (looks for 'keyword' word first)")

In [ ]:
# Process all chunks - FAST MODE with skip logic
all_keywords = {}
keyword_locations = {}

print("="*70)
print(f"Processing {num_chunks} chunks - FAST MODE ⚡")
print("="*70)

start_time = time.time()
transcribed_count = 0
skipped_count = 0

for i in range(num_chunks):
    start_min = i * CHUNK_DURATION_MIN
    start_sec = start_min * 60
    duration_sec = CHUNK_DURATION_MIN * 60
    
    chunk_file = f"audio_chunks/chunk_{i:03d}.mp3"
    transcript_file = f"transcriptions/chunk_{i:03d}.txt"
    
    print(f"\n[{i+1}/{num_chunks}] Minutes {start_min}-{start_min+CHUNK_DURATION_MIN}")
    
    # Extract chunk (skip if already exists)
    if not os.path.exists(chunk_file):
        cmd = ['ffmpeg', '-y', '-v', 'quiet', '-ss', str(start_sec),
               '-i', audio_file, '-t', str(duration_sec),
               '-acodec', 'libmp3lame', chunk_file]
        subprocess.run(cmd, check=True)
    
    # OPTIMIZATION: Skip if already transcribed
    if os.path.exists(transcript_file):
        print(f"  ⏭ Skipping transcription (already exists)")
        with open(transcript_file, 'r', encoding='utf-8') as f:
            content = f.read()
            # Extract just the transcript (skip header)
            parts = content.split('='*70)
            transcript = parts[-1].strip() if len(parts) > 1 else content
        skipped_count += 1
    else:
        # Transcribe with FORCED ENGLISH and TINY model (FAST!)
        result = model.transcribe(chunk_file, language='en', verbose=False, fp16=torch.cuda.is_available())
        transcript = result["text"]
        
        # Save transcription
        with open(transcript_file, 'w', encoding='utf-8') as f:
            f.write(f"Chunk {i} - Minutes {start_min}-{start_min+CHUNK_DURATION_MIN}\n")
            f.write("="*70 + "\n\n")
            f.write(transcript)
        
        transcribed_count += 1
    
    # Search for keywords using SMART search
    matches = find_keywords(transcript)
    
    if matches:
        print(f"  ✓✓✓ FOUND {len(matches)} KEYWORD(S)! ✓✓✓")
        for pos_num, letter, phonetic in matches:
            all_keywords[pos_num] = letter
            keyword_locations[pos_num] = {
                'letter': letter,
                'phonetic': phonetic,
                'chunk': i,
                'time_min': start_min
            }
            print(f"      Position {pos_num}: {letter} ({phonetic})")
    else:
        snippet = transcript[:80].replace('\n', ' ')
        print(f"  - No keywords (sample: {snippet}...)")
    
    # Progress every 5 chunks
    if (i + 1) % 5 == 0:
        elapsed = (time.time() - start_time) / 60
        avg_time = elapsed / (i + 1) if (i + 1) > 0 else 0
        remaining = avg_time * (num_chunks - i - 1)
        print(f"\n  ⏱ Progress: {i+1}/{num_chunks} | {elapsed:.1f}m elapsed | ~{remaining:.1f}m remaining")
        print(f"  🔑 Keywords found: {len(all_keywords)} | Transcribed: {transcribed_count} | Skipped: {skipped_count}")

total_time = (time.time() - start_time) / 60
print(f"\n{'='*70}")
print(f"✓ Completed in {total_time:.1f} minutes")
print(f"✓ Transcribed {transcribed_count} chunks, skipped {skipped_count}")
print(f"✓ Audio chunks saved to: audio_chunks/")
print(f"✓ Transcriptions saved to: transcriptions/")
print(f"{'='*70}")

In [ ]:
# Display final results
print("\n" + "="*70)
print("FINAL RESULTS")
print("="*70)

if all_keywords:
    print(f"\n✓ Found {len(all_keywords)} keyword letters:\n")
    
    for pos in sorted(all_keywords.keys()):
        loc = keyword_locations[pos]
        print(f"  Position {pos}: {loc['letter']} ({loc['phonetic']}) - at {loc['time_min']} min")
    
    # Build keyword
    max_pos = max(all_keywords.keys())
    keyword = ""
    for i in range(1, max_pos + 1):
        keyword += all_keywords.get(i, "_")
    
    print(f"\n{'='*70}")
    print(f"🚩 KEYWORD: {keyword}")
    print(f"{'='*70}")
    
    # Check completeness
    missing = [i for i in range(1, max_pos + 1) if i not in all_keywords]
    if missing:
        print(f"\n⚠ Missing positions: {missing}")
        print("\nTo manually search transcriptions for missing letters:")
        print("  !grep -i 'keyword' transcriptions/*.txt")
    else:
        print(f"\n✓✓✓ COMPLETE KEYWORD! ✓✓✓")
    
    # Save solution
    with open('SOLUTION.txt', 'w') as f:
        f.write(f"KEYWORD: {keyword}\n\n")
        f.write("Letters found:\n")
        for pos in sorted(all_keywords.keys()):
            loc = keyword_locations[pos]
            f.write(f"  Position {pos}: {loc['letter']} ({loc['phonetic']}) at {loc['time_min']} min\n")
        if missing:
            f.write(f"\nMissing positions: {missing}\n")
    
    print("\n✓ Solution saved to SOLUTION.txt")
    
    # Download result
    from google.colab import files
    files.download('SOLUTION.txt')
    print("\n✓ Downloaded SOLUTION.txt to your computer!")
else:
    print("\n⚠ No keywords found. Manually search transcriptions:")
    print("  !grep -i 'keyword' transcriptions/*.txt")

## Manual Search (if needed)

If some keywords are missing, search manually:

In [ ]:
# Manual search for "keyword" in all transcriptions
!grep -i 'keyword' transcriptions/*.txt

In [ ]:
# Download all transcriptions as zip
!zip -r transcriptions.zip transcriptions/
from google.colab import files
files.download('transcriptions.zip')